# Custom User Data Generation Script

- To stimulate the E-Commerce data, A real-world like data is created using multiple conditions and varoius parameters. 
- In this a interest of user was prdefined in order to create relations between users interaction and purchase history. 
- But the interst was not used for further processing and creating of Recommendation System.

In [1]:
!pip install Faker

In [2]:
import random
import pandas as pd
from faker import Faker
import numpy as np

In [3]:
num_users = 1000
fake = Faker()

user_profiles = []
indian_states = ['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']

for user_id in range(num_users):
    user_interests = random.sample(['Electronics', 'Clothing', 'Books', 'Beauty', 'Home'], random.randint(1, 3))
    age = int(np.random.normal(39, 10))
    age = max(18, min(age, 60))
    gender = random.choices(['Male','Female','Other'], weights=[0.44,0.39,0.16], k=1)[0]
    status = random.choices(['Free', 'Premium'], weights=[0.64,0.36])[0]

    user_profiles.append({
        'User ID': user_id,
        'Age': age,
        'Gender': gender,
        'Location': random.choice(indian_states),
        'Interests': ', '.join(user_interests),
        'Subscription Status': status
    })

# Defining Product Features by Category
product_features = {
    'Electronics': ['4K resolution', 'Bluetooth connectivity', 'Wireless charging', 'Touchscreen display'],
    'Clothing': ['100% cotton', 'Machine washable', 'Breathable fabric', 'Waterproof'],
    'Books': ['Hardcover', 'Paperback', 'Audiobook available', 'E-book available'],
    'Beauty': ['Cruelty-free', 'Paraben-free', 'Organic ingredients', 'Hypoallergenic'],
    'Home': ['Energy-efficient', 'Smart home compatible', 'Easy to assemble', 'Durable']
}

# Generating Product Data
product_categories = ['Electronics', 'Clothing', 'Books', 'Beauty', 'Home']
brands = {category: [fake.company() for _ in range(5)] for category in product_categories}
product_data = []
for category in product_categories:
    num_products = random.randint(10, 30)
    for _ in range(num_products):
        product_name = fake.word() + ' ' + category
        features = ', '.join(random.sample(product_features[category], random.randint(1, len(product_features[category]))))
        availability = random.choices(['In Stock', 'Out of Stock'], weights=[0.8, 0.2])[0]
        brand = random.choice(brands[category])
        product_data.append({
            'Product ID': fake.unique.random_number(),
            'Product Name': product_name,
            'Category': category,
            'Price': random.uniform(10, 500),
            'Brand': brand,
            'Features': features,
            'Average Rating': round(random.uniform(1, 5), 2),
            'Number of Ratings': random.randint(10, 500),
            'Availability': availability
        })


# Generating User Interaction History
interactions = []
interaction_id = 0
for user in user_profiles:
    num_interactions = random.randint(5, 50)
    for _ in range(num_interactions):
        user_interests = user['Interests']
        relevant_products = [product for product in product_data if product['Category'] in user_interests]
        non_relevant_products = [product for product in product_data if product['Category'] not in user_interests]
        if relevant_products:
            # 90% chance to interact with a relevant product, 10% chance to interact with a non-relevant product
            if random.random() < 0.9:
                product = random.choice(relevant_products)
            else:
                product = random.choice(non_relevant_products)
            interaction_type = random.choices(['Viewed', 'Purchased', 'Added to Cart'], weights=[0.45, 0.25, 0.3])[0]
            interactions.append({
                'Interaction ID': interaction_id,
                'User ID': user['User ID'],
                'Product ID': product['Product ID'],
                'Interaction Type': interaction_type,
                'Interaction Timestamp': fake.date_time_this_year(),
            })
            interaction_id += 1

            
# Generating User Purchase History
purchases = []
for interaction in interactions:
    if interaction['Interaction Type'] == 'Purchased':
        user_id = interaction['User ID']
        product_id = interaction['Product ID']
        rating = round(random.uniform(1, 5), 2)
        purchases.append({
            'Purchase ID': fake.unique.random_number(),
            'User ID': user_id,
            'Product ID': product_id,
            'Purchase Timestamp': fake.date_time_this_year(),
            'Rating': rating
        })
        # Updating average rating of the product
        product = next(product for product in product_data if product['Product ID'] == product_id)
        product['Average Rating'] = round((product['Average Rating'] * product['Number of Ratings'] + rating) / (product['Number of Ratings'] + 1), 2)
        product['Number of Ratings'] += 1


In [4]:
# Converting data to dataframes
user_profiles_df = pd.DataFrame(user_profiles)
product_data_df = pd.DataFrame(product_data)
interactions_df = pd.DataFrame(interactions)
purchases_df = pd.DataFrame(purchases)

In [5]:
# Saving data to CSV files
user_profiles_df.to_csv('user.csv', index=False)
product_data_df.to_csv('product.csv', index=False)
interactions_df.to_csv('interactions.csv', index=False)
purchases_df.to_csv('purchases.csv', index=False)